# United States Education and Occupation Information
This extracts the all information in the 2013-2017 American Community Survey PUMS Data into a pandas data frame. Then key variables are obtained to gather aggregate information on what are the wages (WAGP) of the people of the US based on their level of education (SCHL), field of degree (FOD1P) and occupation (OCCP).

## Variables Used:
Wages (WAGP)<br>
Level of Education (SCHL)<br>
Field of Degree (FOD1P)<br>
Occupation (OCCP)<br>
Sex (SEX)<br>
Looking for Job (NWLK)<br>
### Dependencies:

In [1]:
# Dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


## Reading and Organizing the Data
The data is downloaded as a csv from US Census Bureau as a CSV.<br>
too<br>
Data can be downloaded here:<br>https://www2.census.gov/programs-surveys/acs/data/pums/2017/5-Year/<br>
It is then read in to a Pandas DataFrame<br>
The columns are coded according the 2013-2017 ACS PUMS Data Dictionary<br>
Data Dictionary can be found here:<br>
https://www2.census.gov/programs-surveys/acs/tech_docs/pums/data_dict/PUMS_Data_Dictionary_2013-2017.pdf?#

In [2]:
#Loading US Data csv files

# file_to_load = ['data_files/psam_pusa.csv'
#                 ,'data_files/psam_pusb.csv'
#                 ,'data_files/psam_pusc.csv'
#                 ,'data_files/psam_pusd.csv']
# chunksize = 100000

In [3]:
#obtaining the desired columns from the Large Dataset into smaller CSVs

# count = 1
# for path in file_to_load:
#     degree_dataT = pd.read_csv(path, chunksize=chunksize, iterator = True)
#     degree_data = pd.concat(degree_dataT, ignore_index=True)

#     degree_data = degree_data[["FOD1P","OCCP","WAGP","SEX","SCHL","NWLK"]]

#     degree_data.to_csv(f'data_files/usa_{count}.csv')
#     count += 1
#     %reset_selective -f degree

In [4]:
#load US selected variable CSVs
usa_files = ['data_files/usa_1.csv',
            'data_files/usa_2.csv',
            'data_files/usa_3.csv',
            'data_files/usa_4.csv']


us_data1 = pd.read_csv(usa_files[0]).reset_index()
us_data2 = pd.read_csv(usa_files[1]).reset_index()
us_data3 = pd.read_csv(usa_files[2]).reset_index()
us_data4 = pd.read_csv(usa_files[3]).reset_index()

degree_data = pd.concat([us_data1,us_data2,us_data3,us_data4]).drop('index',axis = 1)
degree_data = degree_data.drop('Unnamed: 0',axis = 1)


## Degree Codes

In [5]:
# getting the degree codes and names 
# Converted the information to a csv from the Data Dictionary using Excel

degree_code = pd.read_csv("data_files/degree_codes.csv", header = None)
degree_code = pd.DataFrame(degree_code)
degree_code = degree_code.rename(columns = {0: "degree_code",
                                            1: "degree_name"})
# degree_code.head()

## Degree Counts/Wages

In [6]:
#grouping by degree code
obs = degree_data.groupby('FOD1P')
#getting counts of people for each degree
degree_counts = pd.DataFrame(obs.size())
#getting mean wage for each degree
income = pd.DataFrame(obs.mean()['WAGP'])

#merging frames
income_per_degree = degree_counts.merge(income, on = 'FOD1P')\
                    .reset_index()
#renaming columns
income_per_degree = income_per_degree.rename(columns = {"FOD1P": "degree_code",
                                                        0: "number_of_people",
                                                        "WAGP":"wage/salary"})
# income_per_degree.head()

## Male/Female Wage Data

In [7]:
#grouping degree information by sex
male = degree_data[degree_data['SEX'] == 1].groupby('FOD1P')
female = degree_data[degree_data['SEX'] == 2].groupby('FOD1P')
#obtaining male and female mean wages per degree
male_wage = pd.DataFrame(male.mean()['WAGP'])
female_wage = pd.DataFrame(female.mean()['WAGP'])
                    
#obtaining percentages of male and female per degree
male_perc = pd.DataFrame(male.size())
male_perc = male_perc / degree_counts * 100

female_perc = pd.DataFrame(female.size())
female_perc = female_perc / degree_counts * 100
# merging data into one frame and renaming columns
sex_data = male_perc.merge(male_wage, on = 'FOD1P').reset_index()
sex_data = sex_data.merge(female_perc, on = 'FOD1P')
sex_data = sex_data.merge(female_wage, on = 'FOD1P')

sex_data.columns = ['degree_code',
                    'male_%',
                    'male_wage',
                    'female_%',
                    'female_wage']

sex_data = degree_code.merge(sex_data, on = 'degree_code')
#cleaning data
sex_data = sex_data.drop('degree_code',axis = 1)
sex_data = sex_data.round(2)
#Saving the data to a csv
sex_data.to_csv('data_to_plot/USSexDiffWages.csv', index = False)

sex_data.head()

,degree_name,male_%,male_wage,female_%,female_wage
0,General Agriculture,79.15,41153.71,20.85,34764.45
1,Agriculture Production And Management,77.18,53560.98,22.82,38537.09
2,Agricultural Economics,82.03,62104.21,17.97,47176.20
3,Animal Sciences,50.99,56393.21,49.01,40889.05
4,Food Science,42.80,71772.38,57.20,47489.00


## People Looking for Jobs

In [8]:
#grouping data by degree code only including people that are looking for a job
obs_looking = degree_data[degree_data['NWLK'] == 1].groupby('FOD1P')
#getting counts of people for each degree and renaming column
looking_counts = pd.DataFrame(obs_looking.size()).reset_index()
looking_counts = looking_counts.rename(columns = {"FOD1P": "degree_code",
                                                  0: "looking"})

# looking_counts.head()

## People working on Non-Degree jobs

In [9]:
#gathered the keys for what are considered low_end/non-college jobs (Servers,telemarketers, etc.) 
# Here's our list of low-end jobs (obtained from 2013-2017 ACS PUMS Data Dictionary)

# 4020 .EAT-COOKS
# 4030 .EAT-FOOD PREPARATION WORKERS
# 4040 .EAT-BARTENDERS
# 4050 .EAT-COMBINED FOOD PREPARATION AND SERVING WORKERS,
# .INCLUDING FAST FOOD
# 4060 .EAT-COUNTER ATTENDANTS, CAFETERIA, FOOD CONCESSION, AND
# .COFFEE SHOP
# 4110 .EAT-WAITERS AND WAITRESSES
# 4120 .EAT-FOOD SERVERS, NONRESTAURANT
# 4130 .EAT-MISCELLANEOUS FOOD PREPARATION AND SERVING RELATED
# .WORKERS, INCLUDING DINING ROOM AND CAFETERIA ATTENDANTS AND
# .BARTENDER HELPERS
# 4140 .EAT-DISHWASHERS
# 4150 .EAT-HOSTS AND HOSTESSES, RESTAURANT, LOUNGE, AND COFFEE SHOP
# 4220 .CLN-JANITORS AND BUILDING CLEANERS
# 4230 .CLN-MAIDS AND HOUSEKEEPING CLEANERS
# 4240 .CLN-PEST CONTROL WORKERS
# 4250 .CLN-GROUNDS MAINTENANCE WORKERS
# 4720 .SAL-CASHIERS
# 4740 .SAL-COUNTER AND RENTAL CLERKS
# 4750 .SAL-PARTS SALESPERSONS
# 4760 .SAL-RETAIL SALESPERSONS
# 4940 .SAL-TELEMARKETERS

low_end_jobs = [4020,4030,4040,4050,4060,4110,4120,4130,4140,4150
                ,4220,4230,4240,4250,4720,4740,4750,4760,4940]
data_low_end = degree_data[degree_data["OCCP"].isin(low_end_jobs)]

low_end_counts = pd.DataFrame(data_low_end.groupby('FOD1P').size()).reset_index()
low_end_counts = low_end_counts.rename(columns = {0 : "non-degree jobs",
                                                  "FOD1P": "degree_code",})

# low_end_counts.head()

## Occupation Codes

In [10]:
# getting the Occupation codes and names 
# obatined from 2013-2017 ACS PUMS Data Dictionary

occ_code = pd.read_csv("data_files/Occupation_codes.csv", header = None)
occ_code = pd.DataFrame(occ_code)
occ_code = occ_code.rename(columns = {0: "occupation_code",
                                     1: "occupation_field",
                                     2: "occupation_name"})
occ_code = occ_code.dropna(1)
# occ_code.head()

## Information on Non-College persons

In [11]:
# Using the Data Dictionary, the a range of leve of educations can be obtained

# 12 .Grade 9
# 13 .Grade 10
# 14 .Grade 11
# 15 .12th grade - no diploma
# 16 .Regular high school diploma
# 17 .GED or alternative credential

non_college = list(range(12,18))

# getting all the data for people that are or have not been college students
data_non_college = degree_data[degree_data["SCHL"].isin(non_college)]
#create dataframe
non_college_counts = pd.DataFrame(data_non_college.groupby('OCCP').size()).reset_index()
non_college_wage = pd.DataFrame(data_non_college.groupby('OCCP').mean())[["WAGP"]].reset_index()
non_college_data = non_college_counts.merge(non_college_wage, on = 'OCCP')
non_college_data = non_college_data.rename(columns = {0 : "non-college count",
                                                      "OCCP": "occupation_code",
                                                      "WAGP":"wage/salary"})
#merging occupation codes with salary information
occ_full_data = occ_code.merge(non_college_data, on = "occupation_code")
#cleaning data
occ_full_data = occ_full_data.drop('occupation_code', axis = 1)
occ_full_data = occ_full_data.round(2)
#Saving the data to a csv
occ_full_data.to_csv('data_to_plot/USOccuWages.csv')

occ_full_data.head()

,occupation_field,occupation_name,non-college count,wage/salary
0,MGR,General And Operations Managers,9557,58534.81
1,MGR,Advertising And Promotions Managers,179,43246.37
2,MGR,Marketing And Sales Managers,4459,55627.48
3,MGR,Public Relations And Fundraising Managers,114,50107.89
4,MGR,Administrative Services Managers,1746,47476.61


## Compiling US Education Wages Data

In [12]:
#merge to a final dataframe including all data
full_data = degree_code.merge(income_per_degree, on = 'degree_code')
full_data = full_data.merge(low_end_counts, on = 'degree_code')
full_data = full_data.merge(looking_counts, on = "degree_code")

#finding percentages for people looking for jobs
full_data["looking_%"] = full_data["looking"]\
                                /full_data["number_of_people"]\
                                * 100
#finding percentages of people in non-degree jobs
full_data["NDJ_%"] = full_data["non-degree jobs"]\
                        /full_data["number_of_people"]\
                        * 100
#cleaning data
full_data = full_data.drop('degree_code',axis = 1)
full_data = full_data.round(2)
#Saving the data to a csv
full_data.to_csv('data_to_plot/USWages.csv', index = False)

full_data.head()

,degree_name,number_of_people,wage/salary,non-degree jobs,looking,looking_%,NDJ_%
0,General Agriculture,10170,39821.83,454,163,1.60,4.46
1,Agriculture Production And Management,7770,50132.75,431,159,2.05,5.55
2,Agricultural Economics,3366,59421.07,114,49,1.46,3.39
3,Animal Sciences,10352,48795.40,349,214,2.07,3.37
4,Food Science,2514,57882.36,88,86,3.42,3.50


# Data is now now ready for further analysis/visualization
 CSVs have been created to include all the information to be further analysed, and there are the headers for each CSV:<br>
This information collected for each degree<br>

### USEduWages.csv
degree_name : name of degree<br>
number_of_people : number of people with that degree<br>
wage/salary : wages or salary income past 12 months<br>
non-degree jobs: number of people working in a job not in their field<br>
looking : people looking for jobs <br>
looking_% : percentage of people looking for jobs<br>
NDJ_% : percentage of people working in a job not in their degree<br>

### USOccuWages.csv
occupation_field : abbreviation of the field of the occupation<br>
occupation_name : name of occupation<br>
number_of_people : number of people with that degree<br>
wage/salary : wages or salary income past 12 months<br>

### USSexDiffWages.csv
degree_name	: name of degree<br>
male_% : percentage of male persons<br>
male_wage : count of male persons<br>
female_% : percentage of female persons<br>
female_wage : count of female persons <br>